In [15]:
# imports 
import pandas as pd 
from data_extraction import DataExtractionHelper

helper = DataExtractionHelper() 
pd.set_option('display.max_columns', None)

In [16]:
variant_df = helper.read_in_data('../data/variant_summary.txt', 'txt', '\t')
variant_df

c:\Users\prodb\Documents\Projects\hive\BiomarkerKB\src\data_extraction.py:69: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(input_file, sep = delim)


,#AlleleID,Type,Name,GeneID,GeneSymbol,HGNC_ID,ClinicalSignificance,ClinSigSimple,LastEvaluated,RS# (dbSNP),nsv/esv (dbVar),RCVaccession,PhenotypeIDS,PhenotypeList,Origin,OriginSimple,Assembly,ChromosomeAccession,Chromosome,Start,Stop,ReferenceAllele,AlternateAllele,Cytogenetic,ReviewStatus,NumberSubmitters,Guidelines,TestedInGTR,OtherIDs,SubmitterCategories,VariationID,PositionVCF,ReferenceAlleleVCF,AlternateAlleleVCF
0,15041,Indel,NM_014855.3(AP5Z1):c.80_83delinsTGCTGTAAACTGTA...,9907,AP5Z1,HGNC:22197,Pathogenic,1,-,397704705,-,RCV000000012,"MONDO:MONDO:0013342,MedGen:C3150901,OMIM:61364...",Hereditary spastic paraplegia 48,germline;unknown,germline,GRCh37,NC_000007.13,7,4820844,4820847,na,na,7p22.1,"criteria provided, single submitter",2,-,N,"ClinGen:CA215070,OMIM:613653.0001",3,2,4820844,GGAT,TGCTGTAAACTGTAACTGTAAA
1,15041,Indel,NM_014855.3(AP5Z1):c.80_83delinsTGCTGTAAACTGTA...,9907,AP5Z1,HGNC:22197,Pathogenic,1,-,397704705,-,RCV000000012,"MONDO:MONDO:0013342,MedGen:C3150901,OMIM:61364...",Hereditary spastic paraplegia 48,germline;unknown,germline,GRCh38,NC_000007.14,7,4781213,4781216,na,na,7p22.1,"criteria provided, single submitter",2,-,N,"ClinGen:CA215070,OMIM:613653.0001",3,2,4781213,GGAT,TGCTGTAAACTGTAACTGTAAA
2,15042,Deletion,NM_014855.3(AP5Z1):c.1413_1426del (p.Leu473fs),9907,AP5Z1,HGNC:22197,Pathogenic,1,"Jun 29, 2010",397704709,-,RCV000000013,"MONDO:MONDO:0013342,MedGen:C3150901,OMIM:61364...",Hereditary spastic paraplegia 48,germline,germline,GRCh37,NC_000007.13,7,4827361,4827374,na,na,7p22.1,no assertion criteria provided,1,-,N,"ClinGen:CA215072,OMIM:613653.0002",1,3,4827360,GCTGCTGGACCTGCC,G
3,15042,Deletion,NM_014855.3(AP5Z1):c.1413_1426del (p.Leu473fs),9907,AP5Z1,HGNC:22197,Pathogenic,1,"Jun 29, 2010",397704709,-,RCV000000013,"MONDO:MONDO:0013342,MedGen:C3150901,OMIM:61364...",Hereditary spastic paraplegia 48,germline,germline,GRCh38,NC_000007.14,7,4787730,4787743,na,na,7p22.1,no assertion criteria provided,1,-,N,"ClinGen:CA215072,OMIM:613653.0002",1,3,4787729,GCTGCTGGACCTGCC,G
4,15043,single nucleotide variant,NM_014630.3(ZNF592):c.3136G>A (p.Gly1046Arg),9640,ZNF592,HGNC:28986,Uncertain significance,0,"Jun 29, 2015",150829393,-,RCV000000014,"MONDO:MONDO:0033005,MedGen:C4551772,OMIM:25130...",Galloway-Mowat syndrome 1,germline,germline,GRCh37,NC_000015.9,15,85342440,85342440,na,na,15q25.3,no assertion criteria provided,1,-,N,"OMIM:613624.0001,ClinGen:CA210674,UniProtKB:Q9...",1,4,85342440,G,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4324858,2403615,Duplication,NM_013275.6(ANKRD11):c.2022dup (p.Glu675fs),29123,ANKRD11,HGNC:21316,Pathogenic,1,"Oct 02, 2019",-1,-,RCV003128077,MedGen:C0008073,Developmental disorder,de novo,germline,GRCh38,NC_000016.10,16,89284519,89284520,na,na,16q24.3,"criteria provided, single submitter",1,-,N,-,2,2430112,89284519,C,CT
4324859,2403616,single nucleotide variant,NM_032892.5(FRMD5):c.285G>C (p.Lys95Asn),84978,FRMD5,HGNC:28214,Uncertain significance,0,"Feb 16, 2023",-1,-,RCV003128078,"MedGen:CN322365,OMIM:620094",NEURODEVELOPMENTAL DISORDER WITH EYE MOVEMENT ...,de novo,germline,GRCh37,NC_000015.9,15,44211701,44211701,na,na,15q15.3,"criteria provided, single submitter",1,-,N,-,2,2430113,44211701,C,G
4324860,2403616,single nucleotide variant,NM_032892.5(FRMD5):c.285G>C (p.Lys95Asn),84978,FRMD5,HGNC:28214,Uncertain significance,0,"Feb 16, 2023",-1,-,RCV003128078,"MedGen:CN322365,OMIM:620094",NEURODEVELOPMENTAL DISORDER WITH EYE MOVEMENT ...,de novo,germline,GRCh38,NC_000015.10,15,43919503,43919503,na,na,15q15.3,"criteria provided, single submitter",1,-,N,-,2,2430113,43919503,C,G
4324861,2403617,single nucleotide variant,NM_001165963.4(SCN1A):c.5158A>G (p.Ile1720Val),6323,SCN1A,HGNC:10585,Uncertain significance,0,-,-1,-,RCV003128079,"MONDO:MONDO:0100135,MedGen:C0751122,OMIM:60720...",Severe myoclonic epilepsy in infancy,germline,germline,GRCh37,NC_000002.11,2,166848627,166848627,na,na,-,

In [17]:
variant_df.drop(variant_df[variant_df['Assembly'] == 'GRCh37'].index, inplace = True)
variant_df.shape

(2136789, 34)

In [18]:
# variant_df = pd.concat([variant_df['PhenotypeList'].str.split('|', expand = True).stack().reset_index(level = 1, drop = True).rename('Disease'), variant_df], axis = 1)
# variant_df.drop('PhenotypeList', axis = 1, inplace = True)

variant_df = helper.split_col_on_delim(df = variant_df, split_column = 'PhenotypeList', new_col_name = 'Disease', delim = '|')
variant_df.shape

(2838403, 34)

In [19]:
variant_df = helper.split_col_on_delim(df = variant_df, split_column = 'Disease', new_col_name = 'disease', delim = ';')
variant_df.shape

(3101632, 34)

In [20]:
variant_df = variant_df[variant_df['disease'].str.contains('cancer|carcinoma', regex = True, case = False)]
variant_df.shape

(220938, 34)

In [21]:
variant_df.loc[variant_df['RS# (dbSNP)'] == -1, 'RS# (dbSNP)'] = None
variant_df['RS# (dbSNP)'].value_counts()

5.877787e+08    46
1.154065e+07    44
2.893458e+07    42
7.862011e+08    41
2.893458e+07    39
                ..
1.333599e+09     1
5.592276e+08     1
8.998257e+08     1
9.916833e+08     1
1.748147e+09     1
Name: RS# (dbSNP), Length: 96396, dtype: int64

In [22]:
helper.populate_col(source = variant_df['RS# (dbSNP)'], target_col = 'rs_id', dtype = 'Int64')
helper.biomarker_df.head()

,biomarker_id,main_x_ref,assessed_biomarker_entity,biomarker_status,best_biomarker_type,specimen_type,loinc_code,condition_name,assessed_entity_type,evidence_source,notes,rs_id
13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1800562
207,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,121908830
645,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,137854550
658,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,137854552
660,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,137854560


In [23]:
helper.biomarker_df.reset_index(drop = True, inplace = True)
helper.biomarker_df

,biomarker_id,main_x_ref,assessed_biomarker_entity,biomarker_status,best_biomarker_type,specimen_type,loinc_code,condition_name,assessed_entity_type,evidence_source,notes,rs_id
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1800562
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,121908830
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,137854550
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,137854552
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,137854560
...,...,...,...,...,...,...,...,...,...,...,...,...
220933,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>
220934,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>
220935,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>
220936,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>


In [24]:
helper.format_doid(source = 'rs_id', prefix = 'dbSNP:rs')
helper.biomarker_df.sample(25)

,biomarker_id,main_x_ref,assessed_biomarker_entity,biomarker_status,best_biomarker_type,specimen_type,loinc_code,condition_name,assessed_entity_type,evidence_source,notes,rs_id
160666,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>
72030,NaN,dbSNP:rs377629427,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,377629427
161892,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>
211298,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>
186297,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>
165548,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>
196730,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>
146395,NaN,dbSNP:rs35462442,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35462442
132618,NaN,dbSNP:rs1593078331,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1593078331
76928,NaN,dbSNP:rs1555526610,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1555526610
